# Scraping des offres Indeed (Data / IA) au Maroc

Ce notebook permet de :

- Construire des URLs de recherche Indeed pour plusieurs mots-clés.
- Parcourir **toutes les pages** de résultats pour chaque mot-clé.
- Cliquer sur chaque offre pour récupérer :
  - le titre
  - l'entreprise
  - la localisation
  - les métadonnées (type de contrat, date, etc.)
  - la description
- Sauvegarder le tout dans un fichier JSON : `indeed_stages_data_ia.json`.



Imports & installation

In [17]:
import time
import json
from datetime import datetime
from urllib.parse import quote_plus

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

ModuleNotFoundError: No module named 'undetected_chromedriver'

Fonction pour construire l’URL

In [ ]:
from urllib.parse import quote_plus

def build_search_url(keyword: str, location: str, start: int = 0) -> str:
    """
    Construit l'URL de recherche Indeed.
    - start = index de départ (0, 10, 20, ... pour les pages suivantes)
    """
    base_url = "https://ma.indeed.com/jobs"
    q = quote_plus(keyword)
    loc = quote_plus(location)

    url = f"{base_url}?q={q}&l={loc}"
    if start > 0:
        url += f"&start={start}"   # pagination Indeed

    return url



Fonction principale de scraping

In [ ]:
def scrape_indeed_offers(
    keywords: list[str],
    location: str,
    output_json: str,
    max_offers_per_kw: int | None = None,
):
    """
    Scrape les offres Indeed pour plusieurs mots-clés, sur plusieurs pages.

    - max_offers_per_kw : limite optionnelle du nombre d'offres par mot-clé
                          (toutes pages confondues). None = pas de limite.
    """

    options = uc.ChromeOptions()
    options.add_argument("--no-first-run")
    options.add_argument("--no-service-autorun")
    options.add_argument("--no-default-browser-check")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--start-maximized")

    driver = None
    all_offers = []

    ul_selector = "ul.css-pygyny.eu4oa1w0"
    li_selector = "li.css-1ac2h1w.eu4oa1w0"

    try:
        driver = uc.Chrome(options=options)
        wait = WebDriverWait(driver, 15)

        for kw in keywords:
            print("\n" + "=" * 60)
            print(f"[INFO] Mot-clé de recherche : {kw}")

            offres_kw = 0    # compteur total pour ce mot-clé
            start = 0        # offset de pagination (0, 10, 20, ...)

            # ==========================
            #   BOUCLE SUR LES PAGES
            # ==========================
            while True:
                # si on a une limite, on s'arrête quand elle est atteinte
                if max_offers_per_kw is not None and offres_kw >= max_offers_per_kw:
                    print(f"[INFO] Limite {max_offers_per_kw} atteinte pour '{kw}'.")
                    break

                url = build_search_url(kw, location, start=start)
                print(f"[INFO] Ouverture de l'URL : {url}")
                driver.get(url)
                time.sleep(5)

                # récupérer la liste des offres sur cette page
                try:
                    ul_element = wait.until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ul_selector))
                    )
                    offer_elements = ul_element.find_elements(By.CSS_SELECTOR, li_selector)
                except Exception:
                    print("[WARN] Impossible de trouver les offres sur cette page.")
                    break

                nb_offres_page = len(offer_elements)
                print(f"[INFO] Offres trouvées sur cette page : {nb_offres_page}")

                if nb_offres_page == 0:
                    print("[INFO] Aucune offre, fin de la pagination pour ce mot-clé.")
                    break

                # si on a une limite globale, on ajuste le nombre à traiter
                if max_offers_per_kw is not None:
                    restant = max_offers_per_kw - offres_kw
                    if restant <= 0:
                        break
                    nb_a_traiter = min(nb_offres_page, restant)
                else:
                    nb_a_traiter = nb_offres_page

                # ==========================
                #   BOUCLE SUR LES OFFRES
                # ==========================
                for i in range(nb_a_traiter):
                    print(f"\n[INFO] Traitement offre {offres_kw + 1} pour '{kw}'")

                    # recharger les éléments pour éviter les stale elements
                    try:
                        ul_element = wait.until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, ul_selector))
                        )
                        offer_elements = ul_element.find_elements(
                            By.CSS_SELECTOR, li_selector
                        )
                    except Exception:
                        print("[WARN] Impossible de recharger la liste des offres.")
                        break

                    if i >= len(offer_elements):
                        print("[WARN] Moins d'offres que prévu sur cette page.")
                        break

                    offer_li = offer_elements[i]

                    # scroll + clic
                    driver.execute_script(
                        "arguments[0].scrollIntoView({block: 'center'});", offer_li
                    )
                    time.sleep(1)

                    try:
                        offer_li.click()
                    except Exception as e:
                        print(f"[WARN] Impossible de cliquer sur l'offre : {e}")
                        continue

                    # attendre la right pane
                    try:
                        wait.until(
                            EC.presence_of_element_located(
                                (
                                    By.CSS_SELECTOR,
                                    "div.jobsearch-RightPane.css-6iabie.eu4oa1w0",
                                )
                            )
                        )
                    except Exception:
                        print("[WARN] Right pane non trouvée, on passe à l'offre suivante.")
                        continue

                    time.sleep(2)

                    # ============ Extraction ============
                    try:
                        title = driver.find_element(
                            By.CSS_SELECTOR,
                            "div.jobsearch-JobInfoHeader-title-container",
                        ).text
                    except Exception:
                        title = ""

                    try:
                        company = driver.find_element(
                            By.CSS_SELECTOR,
                            "span.css-qcqa6h.e1wnkr790",
                        ).text
                    except Exception:
                        company = ""

                    try:
                        location_text = driver.find_element(
                            By.CSS_SELECTOR,
                            "div#jobLocationText",
                        ).text
                    except Exception:
                        location_text = ""

                    try:
                        meta = driver.find_element(
                            By.CSS_SELECTOR,
                            "div.jobsearch-JobMetadataFooter",
                        ).text
                    except Exception:
                        meta = ""

                    try:
                        description = driver.find_element(
                            By.CSS_SELECTOR,
                            "div#jobDescriptionText",
                        ).text
                    except Exception:
                        description = ""

                    offer_data = {
                        "search_keyword": kw,
                        "title": title,
                        "company": company,
                        "location": location_text,
                        "meta": meta,
                        "description": description,
                        "extract_date": datetime.now().strftime("%Y-%m-%d"),
                    }

                    all_offers.append(offer_data)
                    offres_kw += 1
                    print(
                        f"[INFO] Offre extraite pour '{kw}'. Total pour ce mot-clé : {offres_kw}"
                    )

                # passer à la page suivante : Indeed utilise des pas de 10
                start += 10   # si tu vois qu'il y a 15 offres par page, mets 15

        # Sauvegarde JSON globale
        with open(output_json, "w", encoding="utf-8") as f:
            json.dump(all_offers, f, ensure_ascii=False, indent=2)

        print("\n[SUCCESS] JSON sauvegardé :", output_json)
        print(f"[INFO] Nombre total d'offres enregistrées : {len(all_offers)}")

    finally:
        if driver is not None:
            try:
                driver.quit()
            except Exception:
                pass



Lancer le scraping

In [ ]:
keywords = [
    "stage data",
    "stage data science",
    "stage intelligence artificielle",
    "data scientist",
    "data analyst",
    "machine learning engineer",
    "AI engineer",
]

scrape_indeed_offers(
    keywords=keywords,
    location="Maroc",
    output_json="indeed_stages_data_ia.json",
    max_offers_per_kw=None,  # None = toutes les offres de toutes les pages
)
